In [1]:
from dstools.spark import init_spark2, active_namenode

clusters = ['rnd-dwh', 'rnd-pr']
namenodes = {c: active_namenode(c) for c in clusters}

session_name='ngwl_sp'
arrow_enabled = False

spark = init_spark2({
    'appName': f"{session_name}",
    'spark.yarn.access.namenodes': namenodes['rnd-dwh'],
    'spark.sql.execution.arrow.enabled': arrow_enabled,
#     'spark.driver.extraClassPath': conf.get('spark.driver.extraClassPath'),
#     'spark.extraListeners': conf.get('spark.extraListeners'),
#     'spark.network.timeout': 100000
})

In [3]:
!unzip ngwl-predict-customer-churn.zip

Archive:  ngwl-predict-customer-churn.zip
  inflating: line_items01/line_items_2020-01-01.csv  
  inflating: line_items01/line_items_2020-01-11.csv  
  inflating: line_items01/line_items_2020-01-21.csv  
  inflating: line_items01/line_items_2020-01-31.csv  
  inflating: line_items02/line_items_2020-02-10.csv  
  inflating: line_items02/line_items_2020-02-20.csv  
  inflating: line_items03/line_items_2020-03-01.csv  
  inflating: line_items03/line_items_2020-03-11.csv  
  inflating: line_items03/line_items_2020-03-21.csv  
  inflating: line_items03/line_items_2020-03-31.csv  
  inflating: line_items04/line_items_2020-04-10.csv  
  inflating: line_items04/line_items_2020-04-20.csv  
  inflating: line_items04/line_items_2020-04-24.csv  
  inflating: line_items04/line_items_2020-04-26.csv  
  inflating: line_items04/line_items_2020-04-28.csv  
  inflating: line_items04/line_items_2020-04-30.csv  
  inflating: line_items05/line_items_2020-05-02.csv  
  inflating: line_items05/line_items_202

In [2]:
!ls

check_data.ipynb  line_items05	misc
line_items01	  line_items06	ngwl-predict-customer-churn.zip
line_items02	  line_items07	sample_submission.csv
line_items03	  line_items08	shipments
line_items04	  messages	train


In [2]:
import pandas as pd

from glob import glob

files = glob('line_items*/*')
items = []
for f in files:
    items.append(pd.read_csv(f))

files = glob('shipments/*')
shipments = []
for f in files:
    shipments.append(pd.read_csv(f))

train_df = pd.read_csv('train/train.csv')

message_df = pd.read_csv('messages/messages.csv')
actions_df = pd.read_csv('messages/actions.csv')

addresses_df = pd.read_csv('misc/addresses.csv')
addresses_df = addresses_df.drop_duplicates()
user_profiles_df = pd.read_csv('misc/user_profiles.csv')
user_profiles_df = user_profiles_df.drop_duplicates()
master_categories_df = pd.read_csv('misc/master_categories.csv')

sample_df  = pd.read_csv('sample_submission.csv')

merged_shipments_df = pd.concat(shipments)
merged_items_df = pd.concat(items)

In [23]:
df_sum = df_train.groupby('order_completed_at').sum()
df_count = df_train.groupby('order_completed_at').count()

df_m = pd.merge(df_sum, df_count, on ='order_completed_at')

df_m['perc_targ'] = df_m['target_x']/df_m['target_y']
df_m[['perc_targ']]


pd.crosstab(df_train['target'], df_train['order_completed_at'], normalize=True,)

order_completed_at,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07
target,,,,,,,
0.0,0.031947,0.036345,0.050773,0.054335,0.085071,0.110210,0.130916
1.0,0.019496,0.029183,0.041322,0.083512,0.103420,0.109081,0.114388


In [25]:
!ls shipments/

shipments2020-01-01.csv  shipments2020-04-30.csv
shipments2020-03-01.csv  shipments2020-06-29.csv


In [38]:
merged_items_df.head()

,shipment_id,master_category_id,created_at,price,discount,item_id,quantity,product_id,brand_name,pricer,replaced,cancelled
0,561809,146.0,2018-08-23 12:59:59,27.190001,0.0,6445048,40,9828,ФрутоНяня,Pricer::PerItem,0,0
1,561809,630.0,2018-08-23 13:01:01,123.000000,0.0,6445101,1,2321,Rich,Pricer::PerItem,0,0
2,578604,631.0,2018-09-07 10:22:44,37.189999,0.0,6664938,12,9503,Lipton,Pricer::PerPack,0,0
3,600744,580.0,2018-09-23 21:38:24,39.900002,0.0,6967992,1,981,Мистраль,Pricer::PerItem,0,0
4,600744,123.0,2018-09-23 21:41:46,195.000000,0.0,6968029,1,86720,Мираторг,Pricer::PerItem,0,0


In [39]:
merged_shipments_df.head()

,user_id,ship_address_id,shipment_id,order_id,order_created_at,order_completed_at,shipment_starts_at,retailer,s.order_state,shipment_state,s.city_name,s.store_id,total_cost,rate,dw_kind,promo_total,total_weight,platform,os,shipped_at
0,4077,4058639,178163,1291559,2017-05-10 10:16:41,2020-03-13 11:57:53,2020-03-15 12:00:00,METRO,canceled,canceled,Москва,1,158.0,0,courier,0.0,27610,web,windows,NaN
1,48861,271117,383088,3015822,2018-02-27 10:48:13,2020-04-24 21:09:15,2020-04-30 07:00:00,METRO,complete,shipped,Москва,3,499.0,0,courier,0.0,87806,web,windows,2020-04-30 12:53:17
2,3119,276189,385477,3019844,2018-03-01 11:58:09,2020-04-25 17:30:52,2020-04-26 12:00:00,METRO,complete,shipped,Москва,14,158.0,0,courier,0.0,23029,web,windows,2020-04-26 11:54:23
3,60931,319599,435693,3095458,2018-04-18 09:45:14,2020-03-15 12:51:48,2020-03-16 20:00:00,METRO,complete,shipped,Москва,8,158.0,5,courier,0.0,8605,app,android,2020-03-16 21:29:10
4,87464,479940,506881,3234709,2018-06-21 12:12:33,2020-03-19 04:43:12,2020-03-25 07:00:00,METRO,complete,shipped,Москва,10,158.0,5,courier,0.0,21323,app,ios,2020-03-25 08:52:31


In [71]:
df_shipment = pd.read_csv('shipments/shipments2020-01-01.csv')

In [95]:
user_profiles_df.head()

,user_id,gender,bdate
0,2226207,NaN,NaN
3,1684318,male,1987-10-11
7,2103797,NaN,NaN
17,2226212,NaN,NaN
18,931514,NaN,NaN


In [40]:
merged_items_df[merged_items_df['cancelled'] == 1].head()

,shipment_id,master_category_id,created_at,price,discount,item_id,quantity,product_id,brand_name,pricer,replaced,cancelled
279,1831341,85.0,2020-03-13 05:28:05,199.899994,0.0,41175561,1,5061296,Без бренда,Pricer::PerKilo,0,1
341,1081356,95.0,2019-06-20 17:49:56,57.889999,0.0,14692173,1,69846,365 дней,Pricer::PerItem,0,1
423,1899310,16.0,2020-03-11 01:39:33,121.000000,0.0,40552780,1,58303,Metro Chef,Pricer::PerItem,0,1
440,1899310,281.0,2020-03-11 02:05:58,249.000000,170.0,40553477,1,80465,Paclan,Pricer::PerItem,0,1
517,2314638,648.0,2020-03-10 07:53:55,209.000000,0.0,40354530,1,5061235,Бухта изобилия,Pricer::PerItem,0,1


In [73]:
print(df_shipment.shape)
pd.merge(df_shipment, addresses_df, left_on ='ship_address_id', right_on = 'id').shape

(141546, 20)


(141546, 22)

In [50]:
gr[gr['phone_id'] > 1].head()

,phone_id
id,
103403,2
126321,2
181263,2
183870,2
203336,2


In [51]:
addresses_df[addresses_df.id == 103403]

,id,phone_id
441683,103403,36234
441684,103403,36234


In [54]:
addresses_df = pd.read_csv('misc/addresses.csv')
addresses_df[addresses_df['id'] == 203336]

,id,phone_id
967174,203336,96556
967175,203336,96556


In [57]:
addresses_df = addresses_df.drop_duplicates()

addresses_df[addresses_df.id == 103403]

,id,phone_id
441683,103403,36234


In [66]:
addresses_df[addresses_df['id'] == 322307]

,id,phone_id


In [70]:
len(merged_shipments_df['s.city_name'].unique())

97

In [ ]:
print(df_shipment.shape)
pd.merge(df_shipment, df_)

# Сбор витрины

In [96]:
user_profiles_df.head()

,user_id,gender,bdate
0,2226207,NaN,NaN
3,1684318,male,1987-10-11
7,2103797,NaN,NaN
17,2226212,NaN,NaN
18,931514,NaN,NaN


In [132]:
user_profiles_df['bdate_'].unique()

array([0, '1987-10-11', '1982-09-24', ..., '14.9.2001', '15.02.1978',
       '1976-12-26'], dtype=object)

In [141]:
from datetime import datetime
import numpy as np


user_profiles_df['bdate_'] = user_profiles_df['bdate']
user_profiles_df['bdate_'] = user_profiles_df['bdate_'].replace(np.nan, 0)
user_profiles_df['age'] = user_profiles_df['bdate_'].apply(lambda x: (datetime.today() -
                                                (datetime.strptime(str(x) ,'%Y-%m-%d') \
                                            if '-' in str(x) else 
                                            datetime.strptime(str(x).replace('/', '.'),'%d.%m.%Y'))).days // 365 \
                                if x != 0 and len(str(x)) == 10 else -1)

In [3]:
# добавим phone_id в shipments

print(merged_shipments_df.shape)
shipments = pd.merge(merged_shipments_df, addresses_df, left_on ='ship_address_id', right_on = 'id')
shipments.head()


(2260686, 20)


,user_id,ship_address_id,shipment_id,order_id,order_created_at,order_completed_at,shipment_starts_at,retailer,s.order_state,shipment_state,s.city_name,s.store_id,total_cost,rate,dw_kind,promo_total,total_weight,platform,os,shipped_at,id,phone_id
0,11019,171906,322307,2156687,2017-08-03 21:25:23,2020-02-18 14:07:00,2020-02-20 07:00:00,METRO,complete,shipped,Москва,21,168.0,0,courier,0.0,30170,web,windows,2020-02-20 08:08:54,171906,78177
1,62278,278832,387023,3021953,2018-03-02 17:22:04,2020-01-03 13:09:26,2020-01-03 17:00:00,METRO,complete,shipped,Москва,10,98.0,0,courier,-150.0,11305,web,windows,2020-01-03 18:10:40,278832,96681
2,905126,468407,388943,3018198,2018-02-28 11:32:47,2020-02-12 12:39:28,2020-02-13 11:00:00,METRO,complete,shipped,Москва,21,98.0,5,courier,0.0,13589,app,ios,2020-02-13 12:33:53,468407,140317
3,21412,61962,421048,3030227,2018-03-07 20:37:27,2020-01-25 11:58:56,2020-01-25 18:00:00,METRO,complete,shipped,Москва,8,158.0,0,courier,0.0,9726,web,mac,2020-01-25 19:55:32,61962,16
4,42110,378297,442659,2923996,2017-12-24 11:19:04,2020-01-07 14:30:44,2020-01-07 19:00:00,METRO,complete,shipped,Москва,2,163.0,0,courier,0.0,30323,web,windows,2020-01-07 19:51:37,378297,60075


In [4]:
shipments.drop(columns='id', inplace=True)

In [5]:
shipments['order_month'] = shipments['order_completed_at'].apply(lambda x: x[:7])

shipments.head()

,user_id,ship_address_id,shipment_id,order_id,order_created_at,order_completed_at,shipment_starts_at,retailer,s.order_state,shipment_state,s.city_name,s.store_id,total_cost,rate,dw_kind,promo_total,total_weight,platform,os,shipped_at,phone_id,order_month
0,11019,171906,322307,2156687,2017-08-03 21:25:23,2020-02-18 14:07:00,2020-02-20 07:00:00,METRO,complete,shipped,Москва,21,168.0,0,courier,0.0,30170,web,windows,2020-02-20 08:08:54,78177,2020-02
1,62278,278832,387023,3021953,2018-03-02 17:22:04,2020-01-03 13:09:26,2020-01-03 17:00:00,METRO,complete,shipped,Москва,10,98.0,0,courier,-150.0,11305,web,windows,2020-01-03 18:10:40,96681,2020-01
2,905126,468407,388943,3018198,2018-02-28 11:32:47,2020-02-12 12:39:28,2020-02-13 11:00:00,METRO,complete,shipped,Москва,21,98.0,5,courier,0.0,13589,app,ios,2020-02-13 12:33:53,140317,2020-02
3,21412,61962,421048,3030227,2018-03-07 20:37:27,2020-01-25 11:58:56,2020-01-25 18:00:00,METRO,complete,shipped,Москва,8,158.0,0,courier,0.0,9726,web,mac,2020-01-25 19:55:32,16,2020-01
4,42110,378297,442659,2923996,2017-12-24 11:19:04,2020-01-07 14:30:44,2020-01-07 19:00:00,METRO,complete,shipped,Москва,2,163.0,0,courier,0.0,30323,web,windows,2020-01-07 19:51:37,60075,2020-01


In [6]:
train_df['order_month'] = train_df['order_completed_at']

In [7]:
df = pd.merge(shipments, train_df, on=['phone_id', 'order_month'], how='left')

In [8]:
df.head()

,user_id,ship_address_id,shipment_id,order_id,order_created_at,order_completed_at_x,shipment_starts_at,retailer,s.order_state,shipment_state,s.city_name,s.store_id,total_cost,rate,dw_kind,promo_total,total_weight,platform,os,shipped_at,phone_id,order_month,order_completed_at_y,target
0,11019,171906,322307,2156687,2017-08-03 21:25:23,2020-02-18 14:07:00,2020-02-20 07:00:00,METRO,complete,shipped,Москва,21,168.0,0,courier,0.0,30170,web,windows,2020-02-20 08:08:54,78177,2020-02,2020-02,0.0
1,62278,278832,387023,3021953,2018-03-02 17:22:04,2020-01-03 13:09:26,2020-01-03 17:00:00,METRO,complete,shipped,Москва,10,98.0,0,courier,-150.0,11305,web,windows,2020-01-03 18:10:40,96681,2020-01,2020-01,0.0
2,905126,468407,388943,3018198,2018-02-28 11:32:47,2020-02-12 12:39:28,2020-02-13 11:00:00,METRO,complete,shipped,Москва,21,98.0,5,courier,0.0,13589,app,ios,2020-02-13 12:33:53,140317,2020-02,2020-02,1.0
3,21412,61962,421048,3030227,2018-03-07 20:37:27,2020-01-25 11:58:56,2020-01-25 18:00:00,METRO,complete,shipped,Москва,8,158.0,0,courier,0.0,9726,web,mac,2020-01-25 19:55:32,16,2020-01,2020-01,1.0
4,42110,378297,442659,2923996,2017-12-24 11:19:04,2020-01-07 14:30:44,2020-01-07 19:00:00,METRO,complete,shipped,Москва,2,163.0,0,courier,0.0,30323,web,windows,2020-01-07 19:51:37,60075,2020-01,2020-01,0.0


In [8]:
df.shape, train_df.shape

((2260686, 24), (1159576, 4))

In [71]:
shipments.head()

,user_id,ship_address_id,shipment_id,order_id,order_created_at,order_completed_at,shipment_starts_at,retailer,s.order_state,shipment_state,...,total_cost,rate,dw_kind,promo_total,total_weight,platform,os,shipped_at,phone_id,order_month
0,4077,4058639,178163,1291559,2017-05-10 10:16:41,2020-03-13 11:57:53,2020-03-15 12:00:00,METRO,canceled,canceled,...,158.0,0,courier,0.0,27610,web,windows,NaN,378928,2020-03
1,48861,271117,383088,3015822,2018-02-27 10:48:13,2020-04-24 21:09:15,2020-04-30 07:00:00,METRO,complete,shipped,...,499.0,0,courier,0.0,87806,web,windows,2020-04-30 12:53:17,96702,2020-04
2,3119,276189,385477,3019844,2018-03-01 11:58:09,2020-04-25 17:30:52,2020-04-26 12:00:00,METRO,complete,shipped,...,158.0,0,courier,0.0,23029,web,windows,2020-04-26 11:54:23,96712,2020-04
3,60931,319599,435693,3095458,2018-04-18 09:45:14,2020-03-15 12:51:48,2020-03-16 20:00:00,METRO,complete,shipped,...,158.0,5,courier,0.0,8605,app,android,2020-03-16 21:29:10,93157,2020-03
4,87464,479940,506881,3234709,2018-06-21 12:12:33,2020-03-19 04:43:12,2020-03-25 07:00:00,METRO,complete,shipped,...,158.0,5,courier,0.0,21323,app,ios,2020-03-25 08:52:31,164080,2020-03


In [10]:
from dstools.ml.transformers import noisy_mean_encoder
nm_enc = noisy_mean_encoder(columns=['retailer', 's.city_name'])

nm_enc.fit(df, df['target'])

TargetCategoryEncoder(builder=functools.partial(<function build_noisy_mean_encoder at 0x7f04fe9cb730>, alpha=100, seed=0),
           columns=['retailer', 's.city_name'], n_jobs=1, true_label=None)

In [11]:
#shipments_part = shipments.head(1000)
shipments = nm_enc.transform(shipments)
shipments['order_time'] = shipments['order_completed_at'].apply(lambda x: int(x[11:13]))
shipments['order_day'] = shipments['order_completed_at'].apply(lambda x: pd.to_datetime(x).day)
shipments['order_dayofweek'] = shipments['order_completed_at'].apply(lambda x: pd.to_datetime(x).dayofweek)
#shipments['order_compl_cr_days'] = shipments.apply(lambda x: (pd.to_datetime(x['order_completed_at'])-pd.to_datetime(x['order_created_at'])).days, axis=1)

shipments = pd.get_dummies(shipments, columns = ['s.order_state', 'platform', 'os', 'shipment_state', 'dw_kind'])
shipments.head()

,user_id,ship_address_id,shipment_id,order_id,order_created_at,order_completed_at,shipment_starts_at,retailer,s.city_name,s.store_id,total_cost,rate,promo_total,total_weight,shipped_at,phone_id,order_month,order_time,order_day,order_dayofweek,s.order_state_canceled,s.order_state_cart,s.order_state_complete,s.order_state_resumed,platform_app,platform_web,os_android,os_ios,os_linux,os_mac,os_other,os_windows,shipment_state_canceled,shipment_state_collecting,shipment_state_pending,shipment_state_ready,shipment_state_ready_to_ship,shipment_state_shipped,shipment_state_shipping,dw_kind_courier,dw_kind_express_delivery,dw_kind_pickup
0,11019,171906,322307,2156687,2017-08-03 21:25:23,2020-02-18 14:07:00,2020-02-20 07:00:00,0.53163,0.50345,21,168.0,0,0.0,30170,2020-02-20 08:08:54,78177,2020-02,14,18,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
1,62278,278832,387023,3021953,2018-03-02 17:22:04,2020-01-03 13:09:26,2020-01-03 17:00:00,0.53163,0.50345,10,98.0,0,-150.0,11305,2020-01-03 18:10:40,96681,2020-01,13,3,4,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
2,905126,468407,388943,3018198,2018-02-28 11:32:47,2020-02-12 12:39:28,2020-02-13 11:00:00,0.53163,0.50345,21,98.0,5,0.0,13589,2020-02-13 12:33:53,140317,2020-02,12,12,2,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0
3,21412,61962,421048,3030227,2018-03-07 20:37:27,2020-01-25 11:58:56,2020-01-25 18:00:00,0.53163,0.50345,8,158.0,0,0.0,9726,2020-01-25 19:55:32,16,2020-01,11,25,5,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0
4,42110,378297,442659,2923996,2017-12-24 11:19:04,2020-01-07 14:30:44,2020-01-07 19:00:00,0.53163,0.50345,2,163.0,0,0.0,30323,2020-01-07 19:51:37,60075,2020-01,14,7,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0


In [26]:
# shipments_part = shipments.head(1000)

# shipments['order_compl_cr_days'] = shipments.apply(lambda x: (pd.to_datetime(x['order_completed_at'])-pd.to_datetime(x['order_created_at'])).days, axis=1)
# shipments['order_ship_compl_hours'] = shipments.apply(lambda x: (pd.to_datetime(x['shipment_starts_at'])-pd.to_datetime(x['order_completed_at'])).seconds/3600, axis=1)

In [12]:
sh_itms = pd.read_csv('shipments_items_features.csv')

ship_itms = sh_itms[['shipment_id', 'cancelled_items', 'cat_diversity', 'replaced_items', 'brand_cnt']]
shipments = pd.merge(shipments, ship_itms, on='shipment_id', how = 'left')

In [13]:
shipments.columns

Index(['user_id', 'ship_address_id', 'shipment_id', 'order_id',
       'order_created_at', 'order_completed_at', 'shipment_starts_at',
       'retailer', 's.city_name', 's.store_id', 'total_cost', 'rate',
       'promo_total', 'total_weight', 'shipped_at', 'phone_id', 'order_month',
       'order_time', 'order_day', 'order_dayofweek', 's.order_state_canceled',
       's.order_state_cart', 's.order_state_complete', 's.order_state_resumed',
       'platform_app', 'platform_web', 'os_android', 'os_ios', 'os_linux',
       'os_mac', 'os_other', 'os_windows', 'shipment_state_canceled',
       'shipment_state_collecting', 'shipment_state_pending',
       'shipment_state_ready', 'shipment_state_ready_to_ship',
       'shipment_state_shipped', 'shipment_state_shipping', 'dw_kind_courier',
       'dw_kind_express_delivery', 'dw_kind_pickup', 'cancelled_items',
       'cat_diversity', 'replaced_items', 'brand_cnt'],
      dtype='object')

In [14]:
sp_shipments = spark.createDataFrame(shipments[['user_id', 'ship_address_id', 'shipment_id', 'order_id',
       'order_created_at', 'order_completed_at', 'shipment_starts_at',
       'retailer', 's.city_name', 's.store_id', 'total_cost', 'rate',
       'promo_total', 'total_weight', 'phone_id', 'order_month',
       'order_time', 'order_day', 'order_dayofweek', 's.order_state_canceled',
       's.order_state_cart', 's.order_state_complete', 's.order_state_resumed',
       'platform_app', 'platform_web', 'os_android', 'os_ios', 'os_linux',
       'os_mac', 'os_other', 'os_windows', 'shipment_state_canceled',
       'shipment_state_collecting', 'shipment_state_pending',
       'shipment_state_ready', 'shipment_state_ready_to_ship',
       'shipment_state_shipped', 'shipment_state_shipping', 'dw_kind_courier',
       'dw_kind_express_delivery', 'dw_kind_pickup', 'cancelled_items',
       'cat_diversity', 'replaced_items', 'brand_cnt']])

In [45]:
sp_shipments = sp_shipments.withColumnRenamed('s.city_name', 'city_name')\
.withColumnRenamed('s.store_id', 'store_id')\
.withColumnRenamed('s.order_state_canceled', 'order_state_canceled')\
.withColumnRenamed('s.order_state_cart', 'order_state_cart')\
.withColumnRenamed('s.order_state_complete', 'order_state_complete')\
.withColumnRenamed('s.order_state_resumed', 'order_state_resumed')

In [159]:
import pyspark.sql.functions as func
sp_shipments = sp_shipments.withColumn('order_coml_cr_diff', func.datediff(col('order_completed_at'), col('order_created_at')))

In [149]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([StructField("user_id", IntegerType(), True),\
                    StructField("gender", StringType(), True),
                    StructField("age", IntegerType(), True)])

sp_user_profiles = spark.createDataFrame(user_profiles_df[['user_id', 'gender', 'age']], schema=schema)

In [160]:
sp_shipments = sp_shipments.join(sp_user_profiles, on='user_id', how='left')

In [166]:
import pyspark.sql.functions as func
from pyspark.sql.functions import countDistinct

shipments_group = sp_shipments.groupby(['phone_id', 'order_month']).agg(countDistinct(col('user_id')).alias('user_id_cnt'),\
                                          countDistinct(col('ship_address_id')).alias('ship_address_id_cnt'),\
                                           countDistinct(col('shipment_id')).alias('shipment_id_cnt'),\
                                           countDistinct(col('order_id')).alias('order_id_cnt'),\
                                           countDistinct(col('store_id')).alias('store_id_cnt'),\
                                           func.min(col('total_cost')).alias('total_cost_min'),\
                                           func.max(col('total_cost')).alias('total_cost_max'),\
                                           func.sum(col('total_cost')).alias('total_cost_sum'),\
                                           func.mean(col('total_cost')).alias('total_cost_mean'),\
                                           func.stddev(col('total_cost')).alias('total_cost_std'),\
                                                                        \
                                            func.min(col('rate')).alias('rate_min'),\
                                           func.max(col('rate')).alias('rate_max'),\
                                           func.mean(col('rate')).alias('rate_mean'),\
                                           func.stddev(col('rate')).alias('rate_std'),\
                                                                        
                                           func.min(col('promo_total')).alias('promo_total_min'),\
                                           func.max(col('promo_total')).alias('promo_total_max'),\
                                           func.sum(col('promo_total')).alias('promo_total_sum'),\
                                           func.mean(col('promo_total')).alias('promo_total_mean'),\
                                           func.stddev(col('promo_total')).alias('promo_total_std'),\
                                                                        
                                           func.min(col('total_weight')).alias('total_weight_min'),\
                                           func.max(col('total_weight')).alias('total_weight_max'),\
                                           func.sum(col('total_weight')).alias('total_weight_sum'),\
                                           func.mean(col('total_weight')).alias('total_weight_mean'),\
                                           func.stddev(col('total_weight')).alias('total_weight_std'),\
                                                                        
                                           func.min(col('order_time')).alias('order_time_min'),\
                                           func.max(col('order_time')).alias('order_time_max'),\
                                           func.mean(col('order_time')).alias('order_time_mean'),\
                                           func.stddev(col('order_time')).alias('order_time_std'),\
                                                                        
                                           func.mean(col('order_day')).alias('order_day_mean'),\
                                           func.stddev(col('order_day')).alias('order_day_std'),\
                                                         \
                                           func.min(col('order_dayofweek')).alias('order_dayofweek_min'),\
                                           func.max(col('order_dayofweek')).alias('order_dayofweek_max'),\
                                           func.mean(col('order_dayofweek')).alias('order_dayofweekt_mean'),\
                                           func.stddev(col('order_dayofweek')).alias('order_dayofweek_std'),\
                                                                        \
                                           func.min(col('retailer')).alias('retailer_min'),\
                                           func.max(col('retailer')).alias('retailer_max'),\
                                           func.mean(col('retailer')).alias('retailer_mean'),\
                                           func.stddev(col('retailer')).alias('retailer_std'),\
                                                                        \
                                           func.min(col('city_name')).alias('city_name_min'),\
                                           func.max(col('city_name')).alias('city_name_max'),\
                                           func.mean(col('city_name')).alias('city_name_mean'),\
                                           func.stddev(col('city_name')).alias('city_name_std'),\
                                                                                                     \
                                           func.min(col('cancelled_items')).alias('cancelled_items_min'),\
                                           func.max(col('cancelled_items')).alias('cancelled_items_max'),\
                                           func.sum(col('cancelled_items')).alias('cancelled_items_sum'),\
                                           func.mean(col('cancelled_items')).alias('cancelled_items_mean'),\
                                           func.stddev(col('cancelled_items')).alias('cancelled_items_std'),\
                                                                                                     \
                                           func.min(col('cat_diversity')).alias('cat_diversity_min'),\
                                           func.max(col('cat_diversity')).alias('cat_diversity_max'),\
                                           func.mean(col('cat_diversity')).alias('cat_diversity_mean'),\
                                           func.stddev(col('cat_diversity')).alias('cat_diversity_std'),\
                                                                                                     \
                                           func.min(col('replaced_items')).alias('replaced_items_min'),\
                                           func.max(col('replaced_items')).alias('replaced_items_max'),\
                                           func.sum(col('replaced_items')).alias('replaced_items_sum'),\
                                           func.mean(col('replaced_items')).alias('replaced_items_mean'),\
                                           func.stddev(col('replaced_items')).alias('replaced_items_std'),\
                                                                                                     \
                                           func.min(col('brand_cnt')).alias('brand_cnt_min'),\
                                           func.max(col('brand_cnt')).alias('brand_cnt_max'),\
                                           func.mean(col('brand_cnt')).alias('brand_cnt_mean'),\
                                           func.stddev(col('brand_cnt')).alias('brand_cnt_std'),\
                                                                        \
                                           (func.sum(col('order_state_canceled'))/func.count(col('order_state_canceled'))).alias('order_state_canceled'),\
                                           (func.sum(col('order_state_complete'))/func.count(col('order_state_complete'))).alias('order_state_complete'),\
                                           (func.sum(col('order_state_cart'))/func.count(col('order_state_cart'))).alias('order_state_cart'),\
                                           (func.sum(col('shipment_state_collecting'))/func.count(col('shipment_state_collecting'))).alias('shipment_state_collecting'),\
                                           (func.sum(col('shipment_state_ready_to_ship'))/func.count(col('shipment_state_ready_to_ship'))).alias('shipment_state_ready_to_ship'),\
                                           (func.sum(col('shipment_state_pending'))/func.count(col('shipment_state_pending'))).alias('shipment_state_pending'),\
                                           (func.sum(col('dw_kind_pickup'))/func.count(col('dw_kind_pickup'))).alias('dw_kind_pickup'),\
                                           (func.sum(col('dw_kind_express_delivery'))/func.count(col('dw_kind_express_delivery'))).alias('dw_kind_express_delivery'),\
                                           (func.sum(col('order_state_resumed'))/func.count(col('order_state_resumed'))).alias('order_state_resumed'),\
                                           (func.sum(col('platform_app'))/func.count(col('platform_app'))).alias('platform_app'),\
                                           (func.sum(col('platform_web'))/func.count(col('platform_web'))).alias('platform_web'),\
                                           (func.sum(col('os_android'))/func.count(col('os_android'))).alias('os_android'),\
                                           (func.sum(col('os_ios'))/func.count(col('os_ios'))).alias('os_ios'),\
                                            (func.sum(col('os_mac'))/func.count(col('os_mac'))).alias('os_mac'),\
                                            (func.sum(col('os_windows'))/func.count(col('os_windows'))).alias('os_windows'),\
                                            (func.sum(col('shipment_state_canceled'))/func.count(col('shipment_state_canceled'))).alias('shipment_state_canceled'),\
                                            (func.sum(col('shipment_state_shipped'))/func.count(col('shipment_state_shipped'))).alias('shipment_state_shipped'),\
                                            (func.sum(col('shipment_state_shipping'))/func.count(col('shipment_state_shipping'))).alias('shipment_state_shipping'),\
                                            (func.sum(col('shipment_state_ready'))/func.count(col('shipment_state_ready'))).alias('shipment_state_ready'),\
                                            (func.sum(col('dw_kind_courier'))/func.count(col('dw_kind_courier'))).alias('dw_kind_courier'))

In [52]:
shipments_group.head()

Row(phone_id=101977, order_month='2020-06', user_id_cnt=1, ship_address_id_cnt=3, shipment_id_cnt=5, order_id_cnt=3, store_id_cnt=3, total_cost_min=0.0, total_cost_max=190.0, total_cost_sum=447.0, total_cost_mean=89.4, total_cost_std=87.89653007940643, rate_min=0, rate_max=5, rate_sum=10, rate_mean=2.0, rate_std=2.7386127875258306, promo_total_min=-300.0, promo_total_max=0.0, promo_total_sum=-300.0, promo_total_mean=-60.0, promo_total_std=134.16407864998737, total_weight_min=19140, total_weight_max=60663, total_weight_sum=165236, total_weight_mean=33047.2, total_weight_std=16079.045360344002, order_time_min=13, order_time_max=17, order_time_sum=70, order_time_mean=14.0, order_time_std=1.7320508075688772, order_day_min=1, order_day_max=27, order_day_sum=48, order_day_mean=9.6, order_day_std=12.198360545581526, order_dayofweek_min=0, order_dayofweek_max=5, order_dayofweek_sum=8, order_dayofweekt_mean=1.6, order_dayofweek_std=2.302172886644268, retailer_min=0.45854068156044825, retailer_m

In [167]:
shipments_group.write.mode('overwrite').saveAsTable('dmdigpr_sb.ovgera12_ngwl_train_shipments')

In [93]:
user_profiles_df.drop_duplicates().head()

,user_id,gender,bdate
0,2226207,NaN,NaN
3,1684318,male,1987-10-11
7,2103797,NaN,NaN
17,2226212,NaN,NaN
18,931514,NaN,NaN


In [56]:
shipments_group.join(spark.createDataFrame(train_df), on=['phone_id', 'order_month']).head()

Row(phone_id=4, order_month='2020-03', user_id_cnt=1, ship_address_id_cnt=1, shipment_id_cnt=1, order_id_cnt=1, store_id_cnt=1, total_cost_min=158.0, total_cost_max=158.0, total_cost_sum=158.0, total_cost_mean=158.0, total_cost_std=nan, rate_min=0, rate_max=0, rate_sum=0, rate_mean=0.0, rate_std=nan, promo_total_min=0.0, promo_total_max=0.0, promo_total_sum=0.0, promo_total_mean=0.0, promo_total_std=nan, total_weight_min=25536, total_weight_max=25536, total_weight_sum=25536, total_weight_mean=25536.0, total_weight_std=nan, order_time_min=23, order_time_max=23, order_time_sum=23, order_time_mean=23.0, order_time_std=nan, order_day_min=30, order_day_max=30, order_day_sum=30, order_day_mean=30.0, order_day_std=nan, order_dayofweek_min=0, order_dayofweek_max=0, order_dayofweek_sum=0, order_dayofweekt_mean=0.0, order_dayofweek_std=nan, retailer_min=0.531630122654692, retailer_max=0.531630122654692, retailer_sum=0.531630122654692, retailer_mean=0.531630122654692, retailer_std=nan, city_name_

In [164]:
shipments_group.join(spark.createDataFrame(train_df), on=['phone_id', 'order_month']).\
write.mode('overwrite').saveAsTable('dmdigpr_sb.ovgera12_ngwl_train_shipments_with_target')

# Test

In [62]:
sample_df = pd.read_csv('sample_submission.csv', sep=';')
sp_test = spark.createDataFrame(sample_df)

In [69]:
sp_test = sp_test.drop('Predicted').withColumnRenamed('id', 'phone_id')

In [168]:
sp_test.join(shipments_group['phone_id', 'order_month'].sort('phone_id', desc('order_month')), on = 'phone_id')\
.groupby('phone_id').agg(func.first('order_month').alias('order_month'))\
.join(shipments_group, on=['phone_id', 'order_month'])\
.write.mode('overwrite').saveAsTable('dmdigpr_sb.ovgera12_ngwl_test_shipments_with_target')

In [ ]:
shipments_group.head()

In [31]:
!ls 

check_data-Copy1.ipynb	messages
line_items01		misc
line_items02		ngwl-predict-customer-churn.zip
line_items03		sample_submission.csv
line_items04		shipments
line_items05		shipments_items_features.csv
line_items06		train
line_items07		Untitled.ipynb
line_items08


In [38]:
#sh_itms.groupby('shipment_id').count()

In [56]:
shipments_part.columns

Index(['user_id', 'ship_address_id', 'shipment_id', 'order_id',
       'order_created_at', 'order_completed_at', 'shipment_starts_at',
       'retailer', 's.city_name', 's.store_id', 'total_cost', 'rate',
       'promo_total', 'total_weight', 'shipped_at', 'phone_id', 'order_month',
       's.order_state_canceled', 's.order_state_complete',
       's.order_state_resumed', 'platform_app', 'platform_web', 'os_android',
       'os_ios', 'os_mac', 'os_windows', 'shipment_state_canceled',
       'shipment_state_shipped', 'dw_kind_courier'],
      dtype='object')

In [21]:
shipments.columns

Index(['user_id', 'ship_address_id', 'shipment_id', 'order_id',
       'order_created_at', 'order_completed_at', 'shipment_starts_at',
       'retailer', 's.order_state', 'shipment_state', 's.city_name',
       's.store_id', 'total_cost', 'rate', 'dw_kind', 'promo_total',
       'total_weight', 'platform', 'os', 'shipped_at', 'phone_id',
       'order_month'],
      dtype='object')

In [86]:
shipments['retailer'].unique()

#'s.order_state', 'platform', 'os', 'shipment_state', 'dw_kind'

array(['METRO', 'Ашан', 'Лента', 'ВкусВилл', 'Азбука Вкуса', 'ВИКТОРИЯ',
       'МЕГАМАРТ', 'Командор', 'Бахетле', 'ЗЕЛЕНОЕ ЯБЛОКО', 'Аллея',
       'REAL', 'Хороший', 'ДЕЯ', 'МАГНОЛИЯ', 'Дочки-Сыночки', 'SELGROS',
       'BILLA', 'ВЕГА', 'Быстроном', 'Осень', 'ТД ХОРОШИЙ',
       'ТЦ "Новомариинский" ', 'ТЦ STOЛИЦА', 'СберМаркет', 'КАРАВАН',
       'ИДЕЯ', 'Столичный', 'СЕДЬМАЯ СТОЛИЦА', 'Шан',
       'Забайкальский Привозъ', 'Фреш25', 'АУТЛЕТ', 'ТИТАН',
       'УДАЧНАЯ ПОКУПКА', 'ПАРУС', 'СИГМА', 'КИТ', 'YVES ROCHER',
       'ПроВино', 'ЮгОпт', 'БЕГЕМАГ', 'АГРОКОМПЛЕКС', 'Home Market',
       'Улыбка Радуги', 'МаксМаркет'], dtype=object)

In [ ]:
pd.merge(merged_shipments_df, merged_items_df, on='shipment_id')